### Data Model
1. Data Export : CSV -> JSON
2. Data Import : JSON -> CSV 
3. Create test train spit
4. Write TFRecord : JSON -> TFRecord
5. Data import for Schneider
6. Check data

##### Data Exports

In [1]:
import pandas as pd
import csv, os, sys
import operator
import tensorflow as tf
import json
sys.path.append('/Users/manishrai/Desktop/UMN/Research/Zooniverse/Code/tensorflow/models/research/object_detection/')
from utils import dataset_util

In [2]:
Project_filepath = "/Users/manishrai/Desktop/UMN/Research/Zooniverse/"

CaptureEventID - Consensus Data <br>
Metadata {SiteID, Location, Datetime} - Consensus Data <br>
Images [{ path} - all_images, dimension - GoldStandardBoundBoxCoord] <br>
Observation [bounding box] - GoldStandardBoundBoxCoord

##### Helper functions

In [3]:
"""This function reads a raw image, resizes it with aspect ratio preservation and returns the byte string"""
from PIL import Image
import numpy as np
import io



def resize_jpeg(image,  max_side):
    """ Take Raw JPEG resize with aspect ratio preservation
         and return bytes
    """
    img = Image.open(image)
    img.thumbnail([max_side, max_side], Image.ANTIALIAS)
    b = io.BytesIO()
    img.save(b, 'JPEG')
    image_bytes = b.getvalue()
    return image_bytes

In [4]:
def class_text_to_int(row_label):

    if row_label == 'Aardwolf': return 1
    elif row_label == 'Aardvark': return 2
    elif row_label == 'Baboon': return 3
    elif row_label == 'Buffalo': return 4
    elif row_label == 'BuffaloYouth': return 5
    elif row_label == 'BuffCrestedBustard': return 6
    elif row_label == 'Bushbuck': return 7
    elif row_label == 'CatteEgret': return 8
    elif row_label == 'Cheetah': return 9
    elif row_label == 'DikDik': return 10
    elif row_label == 'DistantAnimal': return 11
    elif row_label == 'Eland': return 12
    elif row_label == 'Elephant': return 13
    elif row_label == 'ElephantYouth': return 14
    elif row_label == 'GazelleGrants': return 15
    elif row_label == 'GazelleGrantsFemale': return 16
    elif row_label == 'GazelleGrantsMale': return 17
    elif row_label == 'GazelleThomsons': return 18
    elif row_label == 'GazelleThomsonsFemale': return 19
    elif row_label == 'GazelleThomsonsMale': return 20
    elif row_label == 'GazelleThomsonsYouth': return 21
    elif row_label == 'Giraffe': return 22
    elif row_label == 'GreyBackedFiscal': return 23
    elif row_label == 'GuineaFowl': return 24
    elif row_label == 'GuineaFowlYouth': return 25
    elif row_label == 'Hare': return 26
    elif row_label == 'Hartebeest': return 27
    elif row_label == 'Hippopotamus': return 28
    elif row_label == 'HotAirBalloon': return 29
    elif row_label == 'Human': return 30
    elif row_label == 'HyenaSpotted': return 31
    elif row_label == 'Impala': return 32
    elif row_label == 'ImpalaFemale': return 33
    elif row_label == 'ImpalaMale': return 34
    elif row_label == 'ImpalaYouth': return 35
    elif row_label == 'Jackal': return 36
    elif row_label == 'KoriBustard': return 37
    elif row_label == 'Leopard': return 38
    elif row_label == 'LionFemale': return 39 
    elif row_label == 'LionMale': return 40
    elif row_label == 'Mongoose': return 41
    elif row_label == 'Ostrich': return 42
    elif row_label == 'OtherBird': return 43
    elif row_label == 'Oxpecker': return 44
    elif row_label == 'Porcupine': return 45
    elif row_label == 'Reedbuck': return 46
    elif row_label == 'ReedbuckMale': return 47
    elif row_label == 'ReedbuckFemale': return 48
    elif row_label == 'ReedbuckYouth': return 49
    elif row_label == 'Rhinoceros': return 50
    elif row_label == 'Rodent': return 51
    elif row_label == 'SecretaryBird': return 52
    elif row_label == 'Serval': return 53
    elif row_label == 'SpottedHyena': return 54
    elif row_label == 'Stork': return 55
    elif row_label == 'SuperbStarling': return 56
    elif row_label == 'Topi': return 57
    elif row_label == 'TopiMale': return 58
    elif row_label == 'TopiFemale': return 59
    elif row_label == 'Truck': return 60
    elif row_label == 'VervetMonkey': return 61
    elif row_label == 'Warthog': return 62
    elif row_label == 'Waterbuck': return 63
    elif row_label == 'WattledStarling': return 64
    elif row_label == 'WhiteHeadBuffaloWeaver': return 65
    elif row_label == 'Wildebeest': return 66
    elif row_label == 'WildebeestYouth': return 67
    elif row_label == 'Zebra': return 68
    elif row_label == 'ZebraYouth': return 69
    else: print('unlabelled!', row_label)

In [40]:
%%writefile data_model.py

def fetchcolvalue(filename, searchkey, output_col_no):
    csvfile = open(os.path.join(Project_filepath, filename), 'r')
    csvdata = csv.reader(csvfile, delimiter=',')
    first_row = next(csvdata)
    ct = 0
    for row in csvdata:
        if searchkey in row[ct]:
            colVal = row[output_col_no]
            break
        else:
            colVal = ""
            
    csvfile.close()
    return colVal

# URL fetching
def fetchurl(filename, searchkey):
    csvfile = open(os.path.join(Project_filepath, filename), 'r')
    csvdata = csv.reader(csvfile, delimiter=',')
    first_row = next(csvdata)
    csvdata2 = sorted(csvdata, key=lambda row: (0, 1)) 
    for row in csvdata2:
        if row[0] == searchkey:
            URL = 'https://snapshotserengeti.s3.msi.umn.edu/' + row[1]
            break
        else:
            URL = ''
    
    csvfile.close()
    return URL

def csvtodict(Project_filepath, bb_data, concensus_data, all_images_data, images):
    lst = []
    event_dict = {}
    csvfile = open(os.path.join(Project_filepath, bb_data), 'r')
    csvdata = csv.reader(csvfile, delimiter=',')
    first_row = next(csvdata)
    for row in csvdata:
        if row[0][0:10] not in event_dict:
            event_dict[row[0][0:10]] = {'metadata' : {"SiteID": fetchcolvalue(concensus_data, row[0][0:10], 3), 
                                  "DateTime": fetchcolvalue(concensus_data, row[0][0:10], 2),  
                                  "Season": fetchcolvalue(all_images_data, row[0][0:10], 1)[0:2]},
                                    'images' : [{"Path" : os.path.join(Project_filepath, 'Data/Schneider_Data/images/', row[0]),
                                "URL" : fetchurl(all_images_data, row[0][0:10]),
                                "dim_x" : fetchcolvalue(bb_data, row[0], 1),
                                "dim_y" : fetchcolvalue(bb_data, row[0], 2),
                                "image_label" : "tbd",
                                'observations' : [{"bb_xmin" : fetchcolvalue(bb_data, row[0], 4),
                                      "bb_ymin" : fetchcolvalue(bb_data, row[0], 5),
                                      "bb_xmax" : fetchcolvalue(bb_data, row[0], 6),
                                      "bb_ymax" : fetchcolvalue(bb_data, row[0], 7), 
                                      "bb_primary_label" : fetchcolvalue(bb_data, row[0], 3), 
                                      "bb_label" : {"species" : fetchcolvalue(bb_data, row[0], 3),
                                                    "pose" : "standing/ sitting/ running"
                                                   }
                                     }]
                               }]
                                    }
    return event_dict


def dicttojson(event_dict):
    with open('event_dict.json', 'w') as outfile:
        json.dump(event_dict, outfile)
    #return json.dump(event_dict, outfile)

def jsontodict(event_json):
    with open(event_json, 'r') as f:
        return json.load(f)


def create_tf_example(data):
    # TODO(user): Populate the following variables from your example.
    height = int(data['images'][0]['dim_y']) # Image height
    width = int(data['images'][0]['dim_x']) # Image width
    filename = str(data['images'][0]['Path']) # Filename of the image. Empty if image is not from file
    encoded_image_data = resize_jpeg(str(data['images'][0]['Path']),  10) # Encoded image bytes
    image_format = b'jpg' # b'jpeg' or b'png'

    xmins = [float(data['images'][0]['observations'][0]['bb_xmin'])] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [float(data['images'][0]['observations'][0]['bb_xmax'])] # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = [float(data['images'][0]['observations'][0]['bb_ymin'])] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [float(data['images'][0]['observations'][0]['bb_ymax'])] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    #classes_text = data[''] # List of string class name of bounding box (1 per box)
    #classes = data[''] # List of integer class id of bounding box (1 per box)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_image_data),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      #'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      #'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    
    return tf_example


def encode_to_tfr_record(test_feature, out_tfr_file):
    with tf.python_io.TFRecordWriter(out_tfr_file) as writer:
        for k, v in test_feature.items():
            example = create_tf_example(v)
            writer.write(example.SerializeToString())

Overwriting data_model.py


In [38]:
bb_data = 'Data/Schneider_Data/TestGoldStandardBoundBoxCoord.csv'
concensus_data = 'Data/consensus_data.csv'
all_images_data = 'Data/all_images.csv'
images = 'Data/Schneider_Data/'
event_dict = csvtodict(Project_filepath, bb_data, concensus_data, all_images_data, images) # In this step we create a dictionary from the csv
dicttojson(event_dict) # storing the dictionary to json
event_dict = jsontodict("event_dict.json") # getting back the dictionary. For creating the tfrecord we will use this 
encode_to_tfr_record(event_dict, 'test_event.tfrecord') # this step creates a tfrecord file
event_dict

{u'ASG000clcb': {u'images': [{u'Path': u'/Users/manishrai/Desktop/UMN/Research/Zooniverse/Data/Schneider_Data/images/ASG000clcb.jpg',
    u'URL': u'https://snapshotserengeti.s3.msi.umn.edu/S4/U09/U09_R2/S4_U09_R2_IMAG1737.JPG',
    u'dim_x': u'2048',
    u'dim_y': u'1536',
    u'image_label': u'tbd',
    u'observations': [{u'bb_label': {u'pose': u'standing/ sitting/ running',
       u'species': u'Wildebeest'},
      u'bb_primary_label': u'Wildebeest',
      u'bb_xmax': u'1487',
      u'bb_xmin': u'1457',
      u'bb_ymax': u'675',
      u'bb_ymin': u'648'}]}],
  u'metadata': {u'DateTime': u'2012-04-15 14:37:30',
   u'Season': u'S4',
   u'SiteID': u'U09'}},
 u'ASG000dz24': {u'images': [{u'Path': u'/Users/manishrai/Desktop/UMN/Research/Zooniverse/Data/Schneider_Data/images/ASG000dz24.jpg',
    u'URL': u'https://snapshotserengeti.s3.msi.umn.edu/S4/J01/J01_R1/S4_J01_R1_IMAG0111.JPG',
    u'dim_x': u'2048',
    u'dim_y': u'1536',
    u'image_label': u'tbd',
    u'observations': [{u'bb_label'

In [7]:
for k, v in event_dict.items():
    print create_tf_example(v)

features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\333\000C\000\010\006\006\007\006\005\010\007\007\007\t\t\010\n\014\024\r\014\013\013\014\031\022\023\017\024\035\032\037\036\035\032\034\034 $.\' \",#\034\034(7),01444\037\'9=82<.342\377\333\000C\001\t\t\t\014\013\014\030\r\r\0302!\034!22222222222222222222222222222222222222222222222222\377\300\000\021\010\000\007\000\n\003\001\"\000\002\021\001\003\021\001\377\304\000\037\000\000\001\005\001\001\001\001\001\001\000\000\000\000\000\000\000\000\001\002\003\004\005\006\007\010\t\n\013\377\304\000\265\020\000\002\001\003\003\002\004\003\005\005\004\004\000\000\001}\001\002\003\000\004\021\005\022!1A\006\023Qa\007\"q\0242\201\221\241\010#B\261\301\025R\321\360$3br\202\t\n\026\027\030\031\032%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\203\204\205\206\207\210\211\212\222\223\224\225\226\227\230\231\232\242\243\244\245\246\24

In [8]:
context_features = {
                    "image/filename" : tf.FixedLenFeature([], tf.string),
                    "image/format" : tf.FixedLenFeature([], tf.string),
                    "image/height" : tf.FixedLenFeature([], tf.int64),
                    "image/source_id" : tf.FixedLenFeature([], tf.string),
                    "image/width" : tf.FixedLenFeature([], tf.int64),
                    "image/encoded" :tf.FixedLenFeature([], tf.string)
                    }

sequence_features = {
                     "image/object/bbox/xmin" : tf.FixedLenSequenceFeature([], tf.int64),
                     "image/object/bbox/xmax" : tf.FixedLenSequenceFeature([], tf.int64),
                     "image/object/bbox/ymin" : tf.FixedLenSequenceFeature([], tf.int64),
                     "image/object/bbox/ymax" : tf.FixedLenSequenceFeature([], tf.int64)
                    }


context, sequence = tf.parse_single_sequence_example(serialized='test_event.tfrecord',
                                          context_features=context_features,
                                          sequence_features=sequence_features,
                                          example_name=None,
                                          name=None)

context, sequence

({'image/encoded': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:0' shape=() dtype=string>,
  'image/filename': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:1' shape=() dtype=string>,
  'image/format': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:2' shape=() dtype=string>,
  'image/height': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:3' shape=() dtype=int64>,
  'image/source_id': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:4' shape=() dtype=string>,
  'image/width': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:5' shape=() dtype=int64>},
 {'image/object/bbox/xmax': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:6' shape=(?,) dtype=int64>,
  'image/object/bbox/xmin': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:7' shape=(?,) dtype=int64>,
  'image/object/bbox/ymax': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequence

### Decoding the TFRecord file

In [41]:
dataset = tf.data.Dataset.from_tensor_slices(sequence)

NameError: name 'sequence' is not defined

In [12]:
dataset

<TensorSliceDataset shapes: {image/object/bbox/xmax: (), image/object/bbox/ymin: (), image/object/bbox/xmin: (), image/object/bbox/ymax: ()}, types: {image/object/bbox/xmax: tf.int64, image/object/bbox/ymin: tf.int64, image/object/bbox/xmin: tf.int64, image/object/bbox/ymax: tf.int64}>